# Using n-grams for policies

In [11]:
import warnings
warnings.filterwarnings("ignore")
import os, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense

In [12]:
pickle_jar = '../pickles/nn_models/'
policy_file = 'policy_index_dict.pckl'
policies = pickle.load(open(os.path.join(pickle_jar, policy_file),'rb'))
len(policies)

116

In [2]:
with open('../pickles/df_nlp_no_negative_multilabel_target.pckl', 'rb') as p:
    df = pickle.load(p)
df.head()

,text,target
0,privacy policy this privacy policy (hereafter ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1 about our products 1 1 our products offer a...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,'2 the information we collect the information...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"'2 2 in addition, we store certain information...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,(c) to remember your preferences and registrat...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
list_corpus = df['text'].tolist()
list_labels = df['target'].tolist()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

In [ ]:
#CountVectorizer(analyzer=...'word', binary=False, decode_error=...'strict',
#        dtype=<... 'numpy.int64'>, encoding=...'utf-8', input=...'content',
#        lowercase=True, max_df=1.0, max_features=None, min_df=1,
#        ngram_range=(1, 1), preprocessor=None, stop_words=None,
#        strip_accents=None, token_pattern=...'(?u)\\b\\w\\w+\\b',
#        tokenizer=None, vocabulary=None)

## Vectorize Unigram & Bigrams

In [5]:
vectorizer = CountVectorizer(ngram_range=(1,2)) # (1,1), (1,2), (2,2)

X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [7]:
X_train_counts.shape

(12405, 159776)

In [14]:
y_train_np = np.array(y_train)

In [18]:
input_dim = X_train_counts.shape[1]

nn = Sequential()
nn.add(Dense(116*2, activation="relu", input_shape=(input_dim,)))
nn.add(Dense(116, activation="sigmoid"))
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [19]:
nn.fit(X_train_counts.toarray(), y_train_np)

Epoch 1/10
12405/12405 [==============================] - 1285s - loss: 0.0697 - acc: 0.9912  
Epoch 2/10
12405/12405 [==============================] - 1084s - loss: 0.0128 - acc: 0.9970  
Epoch 3/10
12405/12405 [==============================] - 1117s - loss: 0.0055 - acc: 0.9984  
Epoch 4/10
12405/12405 [==============================] - 1108s - loss: 0.0034 - acc: 0.9992  
Epoch 5/10
12405/12405 [==============================] - 1125s - loss: 0.0023 - acc: 0.9995  
Epoch 6/10
12405/12405 [==============================] - 1046s - loss: 0.0017 - acc: 0.9996  
Epoch 7/10
12405/12405 [==============================] - 1171s - loss: 0.0015 - acc: 0.9997  
Epoch 8/10
12405/12405 [==============================] - 1188s - loss: 0.0014 - acc: 0.9998  
Epoch 9/10
12405/12405 [==============================] - 1033s - loss: 0.0011 - acc: 0.9998  
Epoch 10/10
12405/12405 [==============================] - 957s - loss: 0.0011 - acc: 0.9998   


In [ ]:
y_predicted = nn.predict(X_test_counts.toarray())

In [ ]:
nn_model_path = '../pickles/nn_models/'
#pickle.dump(open(os.path.join(nn_model_path,'bigram_vectorizer.pckl'),'wb'))
#nn.save(os.path.join(nn_model_path,'nn_model_bigrams.h5'))